## Example of Predictive Model

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestRegressor

In [ ]:
ROOT_PATH = pd.read_csv("ROOT_PATH.txt").columns[0]
PROCESSED_PATH = os.path.join(ROOT_PATH, "datasets", "processed")
del(ROOT_PATH)
file_path_train = os.path.join(PROCESSED_PATH, "train.csv")
file_path_test = os.path.join(PROCESSED_PATH, "test.csv")
file_path_sample_submission = os.path.join(PROCESSED_PATH, "sample_submission.csv")
file_path_submission = os.path.join(PROCESSED_PATH, "submission.csv")

In [ ]:
def read_supplementary(filename, parammeter):
    file_path_supplementary_LEAD = os.path.join(PROCESSED_PATH, filename)
    supplementary = pd.read_parquet(file_path_supplementary_LEAD)
    supplementary = supplementary[supplementary['Parameter.Name'] == parammeter]
    supplementary.drop(["Parameter.Name", "X1st.Max.Value"], axis=1, inplace=True)
    supplementary['Date.Local'] = supplementary['Date.Local'].apply(lambda x: x.year)
    supplementary.rename(columns={'Date.Local': 'Year', 'Arithmetic.Mean': parammeter}, inplace=True)
    supplementary = pd.DataFrame(supplementary.groupby(["State.Name","Year"])[parammeter].median())
    return supplementary

In [ ]:
def input_supplementary(input_data):
    df = read_supplementary(input_data.iloc[0].file, input_data.iloc[0].parameter)
    for i in range(1, input_data.shape[0]):
        df1 = read_supplementary(input_data.iloc[i].file, input_data.iloc[i].parameter)
        df = pd.merge(df, df1, on=["State.Name", "Year"], how='outer')
    return df

In [ ]:
def supp_feature(supplementary, state, year, lag):
    supplementary_state = supplementary[supplementary.index.get_level_values('State.Name') == state]
    return pd.concat([pd.Series(index = ['State.Name', 'Year'], data = [state, year]), 
                    supplementary_state[(supplementary_state.index.get_level_values('Year') >= year-lag)&
                               (supplementary_state.index.get_level_values('Year') <= year)] \
    .median()])

In [ ]:
input_data = pd.DataFrame({
    "file" : ["supplementary_NONOxNOy.parquet", "supplementary_LEAD.parquet", "supplementary_VOCS.parquet", 
              "supplementary_VOCS.parquet", "supplementary_HAPS.parquet",
              "supplementary_HAPS.parquet"],
    "parameter": ["Nitric oxide (NO)", "Lead (TSP) STP", "Toluene", 
                  "Total NMOC (non-methane organic compound)", "Benzene", "Ethylene dichloride"]
})

In [ ]:
supplementary = input_supplementary(input_data)

In [ ]:
train = pd.read_csv(file_path_train)

In [ ]:
locations = sorted(set(supplementary.index.map(lambda x: x[0])))

In [ ]:
supplementary = pd.DataFrame([supp_feature(supplementary, loc, y, 10) for loc in locations
                   for y in range(1990, 2019)])

In [ ]:
train_supp = pd.merge(train, supplementary, on=["State.Name", "Year"], how='left')
del(train)
del(locations)

In [ ]:
y = train_supp['Incidence']

In [ ]:
X = train_supp[list({'Nitric oxide (NO)',
       'Lead (TSP) STP', 'Toluene',
       'Total NMOC (non-methane organic compound)', 'Benzene',
       'Ethylene dichloride'})]

In [ ]:
del(train_supp)

In [ ]:
X = X.interpolate(method="spline", order=3)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

In [ ]:
model.fit(X, y)

In [ ]:
test = pd.read_csv(file_path_test)

In [ ]:
test_supp = pd.merge(test, supplementary, on=["State.Name", "Year"], how='left')
del(test)

In [ ]:
X = test_supp[list({'Nitric oxide (NO)',
       'Lead (TSP) STP', 'Toluene',
       'Total NMOC (non-methane organic compound)', 'Benzene',
       'Ethylene dichloride'})]
del(test_supp)

In [ ]:
X = X.interpolate(method="spline", order=3)

In [ ]:
X = X.fillna(0)

In [ ]:
submission = pd.read_csv(file_path_sample_submission)

In [ ]:
submission['Incidence'] = model.predict(X)

In [ ]:
submission.to_csv(file_path_submission, index=False)